In [1]:
import os
import sys

os.environ["TOKENIZERS_PARALLELISM"] = "false"
# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath(os.path.join('..', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Importa la tua libreria
from code_graph_indexer import CodeNavigator, CodeReader, CodeRetriever
from code_graph_indexer.providers.embedding import FastEmbedProvider
from code_graph_indexer.storage.sqlite import SqliteGraphStorage

# 1. SETUP DEL TUO SISTEMA (Come hai fatto nei test)
repo_path = "/Users/filippodaminato/Desktop/test_repos/7f10a3a2e3b9/worktrees/main"
db_path = "/Users/filippodaminato/Documents/sheep-sys/sheep-codebase-indexer/sheep_index.db"

storage = SqliteGraphStorage(db_path)
provider = FastEmbedProvider()

# indexer = CodebaseIndexer(repo_path, storage)

retriever = CodeRetriever(storage, provider)
reader = CodeReader(storage)
navigator = CodeNavigator(storage)


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
repo_id = "ce77bdfc-eb31-4c5e-9e66-426a638173e8"

In [3]:
# 2. DEFINIZIONE DEI TOOL
# Qui "impacchettiamo" i tuoi metodi per l'LLM

def search_tool(query: str):
    """Cerca concetti o codice nella codebase. Ottimo per iniziare l'esplorazione."""
    results = retriever.retrieve(query, repo_id=repo_id, limit=5, strategy="keyword")
    # Convertiamo i risultati in una stringa leggibile per l'LLM
    return "\n".join([r.render() for r in results])

def read_file_tool(file_path: str, start_line: int = None, end_line: int = None):
    """Legge il contenuto di un file. Puoi specificare linee opzionali."""
    res = reader.read_file(repo_id, file_path, start_line, end_line)
    return res['content']

def inspect_dependencies_tool(node_id: str):
    """Trova chi chiama questa funzione/classe (impatto) o cosa viene chiamato (pipeline)."""
    # Combiniamo i metodi del navigator per dare una visione completa
    impact = navigator.analyze_impact(node_id)
    pipeline = navigator.visualize_pipeline(node_id)
    return f"IMPACT (Callers):\n{impact}\n\nPIPELINE (Calls):\n{pipeline}"

In [4]:
print(search_tool("Application Entry point"))